In [1]:
import pandas as pd
import numpy as np
from datetime  import datetime  
from datetime import timedelta  
import lightgbm as lgb
from sklearn import preprocessing, metrics
from sklearn.model_selection import ParameterGrid
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow
import mlflow.lightgbm

In [3]:
data_location = '../00_data/sample_submission.csv'
submission = pd.read_csv(data_location)

In [50]:
data_location = '../04_submissions/best_submission_day1-5.csv'
best_submission = pd.read_csv(data_location)

In [5]:
X_train = pd.read_pickle('../01_preprocessed_data/X_1_train.pkl')
y_train = pd.read_pickle('../01_preprocessed_data/y_1_train.pkl')
X_val = pd.read_pickle('../01_preprocessed_data/X_1_val.pkl')
y_val = pd.read_pickle('../01_preprocessed_data/y_1_val.pkl')
X_test = pd.read_pickle('../01_preprocessed_data/X_1_test.pkl')

In [6]:
X_train.groupby(['demand_type'])['demand_type', 'demand'].agg(['mean']).reset_index()

,demand_type,demand
,,mean
0,erratic,6.358116
1,intermittent,0.672480
2,lumpy,1.704040
3,smooth,6.247199


In [37]:
average_demand = X_train.groupby(['id'])['id', 'demand'].agg(['mean']).reset_index()
average_demand.columns = average_demand.columns.get_level_values(0)
average_demand.columns = ['id', 'mean_demand']

In [38]:
average_demand.shape

(30490, 2)

In [46]:
zero_demand_ids = average_demand.loc[average_demand['mean_demand'] == 0, 'id'].to_list()

In [61]:
best_submission.loc[best_submission['id'].isin(zero_demand_ids) , ["F{}".format(i) for i in range(1, 29)]] = 0

In [62]:
best_submission[best_submission['id'].isin(zero_demand_ids)]

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
4122,HOUSEHOLD_1_518_CA_2_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6048,FOODS_3_778_CA_2_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11667,FOODS_3_296_CA_4_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12791,HOUSEHOLD_1_032_TX_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20514,FOODS_2_394_TX_3_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22279,HOUSEHOLD_1_379_WI_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26276,FOODS_2_057_WI_2_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26751,FOODS_3_135_WI_2_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [63]:
best_submission.to_csv('../04_submissions/best_submission_day1-5_zeros.csv', index = False)